In [ ]:
# python 3
import os
import shutil
import sys
from PIL import Image, ImageFont, ImageDraw
import operator
from fontTools import ttLib

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
cwd = os.getcwd()
print('current working dir: {}'.format(cwd))

In [ ]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' + directory)

In [ ]:
srcFolder = os.path.normpath(os.path.join(cwd, '../fontFiles/TypeTogether'))
desFolder = os.path.normpath(os.path.join(cwd, '../Project04_ImgGen/TypeTogether'))
print('srcFolder path: {}'.format(srcFolder))
print('desFolder path: {}'.format(desFolder))
if not(os.path.exists(srcFolder)):
    print('srcFolder not exists!')
if not(os.path.exists(desFolder)):
    createFolder(desFolder)
    print('desFolder created!')

In [ ]:
def checkDrawSize(draw, drawText, font):
    drawSize = draw.textsize(drawText, font)
    return drawSize
 

In [ ]:
def checkFontDrawSize(draw, fontPath, drawBound, drawText):
    listImgFont = []

    print('checkDrawSize() - drawBound: {}'.format(drawBound))
    for i in range(20, 2001):
        imgFont = ImageFont.truetype(fontPath, i)
        drawSize = draw.textsize(drawText, imgFont)
        listImgFont.append((i, imgFont, drawSize))

        print('checkDrawSize() - fontSize: {}, drawSize; {}'.format(i, drawSize))
        if (drawSize[0] > drawBound[0]) or(drawSize[1] > drawBound[1]):
            break

    i, imgFont, drawSize = listImgFont[len(listImgFont) - 2]
    print('checkDrawSize():return(fontSize: {}, imgFont: {}, drawSize; {}'.format(i, imgFont, drawSize))
    return imgFont, drawSize

In [ ]:
def drawText(draw, fontPath, canvasSize, drawBound, drawText, fillColor, font=None):
    if not isinstance(drawText, list):
        drawText = list([drawText])
        print('drawText() - drawText: {}'.format(drawText))

    imgFont = None
    drawTextSize = len(drawText)
    tempDrawBound = (drawBound[0], drawBound[1] / drawTextSize)
    for i, drawText in enumerate(drawText):
        print('drawText() - i: {}, drawText: {}'.format(i, drawText))
        
        if font != None:
            imgFont = font
            drawSize = drawBound
        else:
            if keepFontSize == True and imgFont != None:
                drawSize = checkDrawSize(draw, drawText, imgFont)
            else:
                imgFont, drawSize = checkFontDrawSize(draw, fontPath, tempDrawBound, drawText)
                
        drawPoint = ((canvasSize[0] - drawSize[0])/2, (canvasSize[1] - drawBound[1])/2 + (tempDrawBound[1] * i) + (tempDrawBound[1] - drawSize[1])/2)
        #draw.multiline_text(drawPoint, drawText, font=imgFont, spacing=0, align="center")
        draw.text(drawPoint, drawText, font=imgFont, fill=fillColor)
    
    print('drawText():return()')
    return True

In [ ]:
def saveImage(destPath, fileName, image, filePrefix):
    fileName = '%s_%s.png' % (filePrefix, fileName)
    filePath = os.path.join(destPath, fileName)
    
    image.save(filePath, "PNG")

In [ ]:
WINDOWS_ENGLISH_IDS = 3, 1, 0x409
WINDOWS_KOREAN_IDS = 3, 1, 0x412
MAC_ROMAN_IDS = 1, 0, 0

FAMILY_RELATED_IDS = dict(
    LEGACY_FAMILY=1,
    TRUETYPE_UNIQUE_ID=3,
    FULL_NAME=4,
    POSTSCRIPT_NAME=6,
    PREFERRED_FAMILY=16,
    WWS_FAMILY=21,
)

def get_full_name(table):
    family_name_rec = None
    for plat_id, enc_id, lang_id in (WINDOWS_ENGLISH_IDS, MAC_ROMAN_IDS):
        for name_id in (
            FAMILY_RELATED_IDS["POSTSCRIPT_NAME"],
            FAMILY_RELATED_IDS["FULL_NAME"],
        ):
            family_name_rec = table.getName(
                nameID=name_id,
                platformID=plat_id,
                platEncID=enc_id,
                langID=lang_id,
            )
            if family_name_rec is not None:
                break
        if family_name_rec is not None:
            break
    if not family_name_rec:
        raise ValueError("family name not found; can't add suffix")
    return family_name_rec.toUnicode()

def get_family_name(table, language):
    family_name_rec = None
    for plat_id, enc_id, lang_id in (language, MAC_ROMAN_IDS):
        for name_id in (
            FAMILY_RELATED_IDS["PREFERRED_FAMILY"],
            FAMILY_RELATED_IDS["LEGACY_FAMILY"],
        ):
            family_name_rec = table.getName(
                nameID=name_id,
                platformID=plat_id,
                platEncID=enc_id,
                langID=lang_id,
            )
            if family_name_rec is not None:
                break
        if family_name_rec is not None:
            break
    if not family_name_rec:
        raise ValueError("family name not found; can't add suffix")
    return family_name_rec.toUnicode()

In [ ]:
def replaceName(familyName, replaceRule):
    for strOld, strNew in replaceRule:
        print('replaceName({}) - strOld: {}, strNew: {}'.format(familyName, strOld, strNew))
        familyName = familyName.replace(strOld, strNew)
        
    print('replaceName():return({})'.format(familyName))
    return familyName        

In [ ]:
def generatePreviewImages(srcPath, destPath, canvasSize, drawBound, drawTextList):
    print('generateTitleImages() - srcPath: {}'.format(srcPath))
    #canvasSize2x = tuple([2 * x for x in canvasSize])
    #drawBound2x = tuple([2 * x for x in drawBound])

    for fileName in os.listdir(srcPath):
        image = Image.new('RGBA', canvasSize, color=(255, 255, 255, 0))
        draw = ImageDraw.Draw(image)
        
        fontFullPath = os.path.join(srcPath, fileName)
        if os.path.isdir(fontFullPath):
            tempDestPath = destPath
            if (keepFolderStructure):
                tempDestPath = os.path.join(destPath, fileName)
                createFolder(tempDestPath)
            generatePreviewImages(fontFullPath, tempDestPath, canvasSize, drawBound, drawTextList)
            continue
        
        try:
            font = ttLib.TTFont(fontFullPath)
        except Exception as ex:
            print('Not a font file!')
            continue
        postScriptName = get_full_name(font['name'])
        font.close()
        
        drawText(draw, fontFullPath, canvasSize, drawBound, drawTextList, (90, 90, 90, 255))
        saveImage(destPath, postScriptName, image, 'preview')

        #imageResize = image.resize(canvasRect, resample=Image.LANCZOS)
        #saveImage(destPath, postScriptName, imageResize, 'preview')
        
        print('generate font preview image... {}, {}'.format(fileName, postScriptName))
        

In [ ]:
def generateTitleImages(srcPath, destPath, canvasSize, drawBound, replaceRule):
    print('generateTitleImages() - srcPath: {}'.format(srcPath))
    canvasSize2x = tuple([2 * x for x in canvasSize])
    drawBound2x = tuple([2 * x for x in drawBound])
    
    for fileName in os.listdir(srcPath):
        image = Image.new('RGBA', canvasSize2x, color=(255, 255, 255, 0))
        draw = ImageDraw.Draw(image)

        fontFullPath = os.path.join(srcPath, fileName)
        if os.path.isdir(fontFullPath):
            tempDestPath = destPath
            if (keepFolderStructure):
                tempDestPath = os.path.join(destPath, fileName)
                createFolder(tempDestPath)
            generateTitleImages(fontFullPath, tempDestPath, canvasSize, drawBound, replaceRule)
            continue
            
        try:
            font = ttLib.TTFont(fontFullPath)
        except Exception as ex:
            print('Not a font file!')
            continue
        postScriptName = get_full_name(font['name'])
        familyName = get_family_name(font['name'], WINDOWS_KOREAN_IDS)
        font.close()
        
        familyName = replaceName(familyName, replaceRule)
        
        drawText(draw, fontFullPath, canvasSize2x, drawBound2x, familyName, (0, 0, 0, 255))
        saveImage(destPath, postScriptName + '@2x', image, 'title')

        imageResize = image.resize(canvasSize, resample=Image.LANCZOS)
        saveImage(destPath, postScriptName, image, 'title')
        
        print('generate font title image... {}, {}'.format(fileName, postScriptName))
        

In [ ]:
def generateContentsImages(srcPath, destPath, canvasHeight, replaceRule):
    print('generateContentsImages() - srcPath: {}'.format(srcPath))
    canvasSize = (5000, canvasHeight)
    
    for fileName in os.listdir(srcPath):
        fontFullPath = os.path.join(srcPath, fileName)
        if os.path.isdir(fontFullPath):
            tempDestPath = destPath
            if (keepFolderStructure):
                tempDestPath = os.path.join(destPath, fileName)
                createFolder(tempDestPath)
            generateContentsImages(fontFullPath, tempDestPath, canvasHeight, replaceRule)
            continue
            
        try:
            font = ttLib.TTFont(fontFullPath)
        except Exception as ex:
            print('Not a font file!')
            continue
        postScriptName = get_full_name(font['name'])
        familyName = get_family_name(font['name'], WINDOWS_KOREAN_IDS)
        font.close()
        
        familyName = replaceName(familyName, replaceRule)

        image = Image.new('RGBA', canvasSize, color=(255, 255, 255, 0))
        draw = ImageDraw.Draw(image)
   
        imgFont, drawSize = checkFontDrawSize(draw, fontFullPath, canvasSize, familyName)
        canvasSize = (drawSize[0], canvasHeight)
        
        image = Image.new('RGBA', canvasSize, color=(255, 255, 255, 0))
        draw = ImageDraw.Draw(image)
      
        drawText(draw, fontFullPath, canvasSize, drawSize, familyName, (0, 0, 0, 255), font=imgFont)
        saveImage(destPath, postScriptName, image, 'contents')
        
        print('generate font contents image... {}, {}'.format(fileName, postScriptName))
        

In [ ]:
keepFolderStructure = True
keepFontSize = True
canvasRect = (296, 146)
drawBound = (280, 90)
drawTextList = ['폰트가 일상이 된다', 'We build fonts!'] #for Sandoll
#drawTextList = ['아름다운 한글디자인', 'New trend fonts!'] #for FontRix
#drawTextList = ['간직하고 싶은 글자', 'Fonts, want to have'] #for TSC
#drawTextList = ['샘물처럼 맑은글씨', 'Life of good font'] #for 좋은글씨
#drawTextList = ['좋은 글자를 그리다', 'Good Typography'] #for 한글씨
#drawTextList = ['바른생각과 바른글씨', 'Right thoughts'] #for 로그인디자인
#drawTextList = ['Fonts from th Netherlands', 'We build fonts!'] #for Typotheque
#drawTextList = ['Joie de vivre aujourd\'hui', 'We build fonts!'] #for ProductionType
#drawTextList = ['High quality fonts and', 'custom type design'] #for TypeTogether
#drawTextList = ['東方文字傳情之美', 'We build fonts!'] #for Arphic

generatePreviewImages(srcFolder, desFolder, canvasRect, drawBound, drawTextList)

In [ ]:
keepFolderStructure = True
canvasSize = (260, 56)
drawBound = (200, 50)
replaceRule = [("Sandoll ", ""), ["Rix", ""], ["TSC", ""], ["GF ", ""], ["HG", ""], ["Log", ""]]

generateTitleImages(srcFolder, desFolder, canvasSize, drawBound, replaceRule)

In [ ]:
keepFolderStructure = True
canvasHeight = 308
replaceRule = [("Sandoll ", ""), ["Rix", ""], ["TSC", ""], ["GF ", ""], ["HG", ""], ["Log", ""]]

generateContentsImages(srcFolder, desFolder, canvasHeight, replaceRule)